In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import DistanceMetric
import os
import sys
sys.path.append('../models')
import ResNet as resnet
import DenseNet as densenet
import EfficientNet as efficientnet
import MobileNetV2 as mobilenet
import ViT as vit

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
##########################################
######### Configure Metric Setting ####### 
##########################################
batch_size = 64
weight_path = './weights/cifar10_resnet18_nothing.pt'
model = torch.load(weight_path)
criterion = nn.CrossEntropyLoss()
member_path = '/media/data1/hyunjun/cifar-10/train/'
nonmember_path = '/media/data1/hyunjun/cifar-10/test/'
data = 'Cifar-10'
##########################################
##########################################

In [4]:
mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) 
if data == 'STL-10':
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
elif data == 'Cifar-10':
    mean, std = (0.491, 0.482, 0.447), (0.247, 0.243, 0.262)
elif data == 'CelebA':
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

model.to(device)
model.eval()

nonmember_set = torchvision.datasets.ImageFolder(root=nonmember_path, transform=trans)
nonmember_loader = DataLoader(nonmember_set, batch_size=batch_size, shuffle=True, drop_last=False,)
member_set = torchvision.datasets.ImageFolder(root=member_path, transform=trans)
member_loader = DataLoader(member_set, batch_size=batch_size, shuffle=True, drop_last=False,)

classes = member_set.classes

In [10]:
latent_vectors = [[] for _ in range(len(classes))]

model.eval()
with torch.no_grad():
    for data in member_loader:
        images, labels = data[0].to(device), data[1].to(device)
        features, _ = model(images)

        deep_features = features.cpu().numpy().tolist()
        actual = labels.cpu().numpy().tolist()
        
        for i in range(len(actual)):
            latent_vectors[actual[i]].append(list(deep_features[i]))


In [ ]:
print("#####################################")
print("### Calculating Cosine Similarity ###")
print("#####################################")

for i in range(len(latent_vectors)):
    cssm = cosine_similarity(np.array(latent_vectors[i]))
    print("class:", classes[i], "max:", np.max(cssm), "min:", np.min(cssm), "mean:", np.mean(cssm), "var:", np.var(cssm))

In [ ]:
print("#####################################")
print("## Calculating Euclidean Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('euclidean')
for i in range(len(latent_vectors)):
    euclidean = dist.pairwise(np.array(latent_vectors[i]))
    print("class:", classes[i], "max:", np.max(euclidean), "min:", np.min(euclidean), "mean:", np.mean(euclidean), "var:", np.var(euclidean))

In [ ]:
print("#####################################")
print("## Calculating Manhattan Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('manhattan')
for i in range(len(latent_vectors)):
    manhattan = dist.pairwise(np.array(latent_vectors[i]))
    print("class:", classes[i], "max:", np.max(manhattan), "min:", np.min(manhattan), "mean:", np.mean(manhattan), "var:", np.var(manhattan))

In [ ]:
print("#####################################")
print("## Calculating Chebyshev Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('chebyshev')
for i in range(len(latent_vectors)):
    chebyshev = dist.pairwise(np.array(latent_vectors[i]))
    print("class:", classes[i], "max:", np.max(chebyshev), "min:", np.min(chebyshev), "mean:", np.mean(chebyshev), "var:", np.var(chebyshev))

In [ ]:
print("#####################################")
print("## Calculating Mahalano. Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('mahalanobis')
for i in range(len(latent_vectors)):
    mahalanobis = dist.pairwise(np.array(latent_vectors[i]))
    print("class:", classes[i], "max:", np.max(mahalanobis), "min:", np.min(mahalanobis), "mean:", np.mean(mahalanobis), "var:", np.var(mahalanobis))